# Задание 1
Преобразуйте таблицу с абсолютными частотностями в семинарской тетрадке в таблицу с tfidf значениями. (Таблица - https://i.ibb.co/r5Nc2HC/abs-bow.jpg) Формула tfidf есть в семинаре на картнике с пояснениями на английском. 
Можно считать в google sheets, екселе, питоне или на листочке. Сделайте скрин получившейся таблицы и загрузите картинку на гитхаб, вставьте ссылку в поле ниже.

https://github.com/deniskapel/Compling_course/blob/master/Tfidf_table.PNG

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB

from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
from nltk.corpus import stopwords

In [2]:
pd.set_option('display.max_colwidth', 500)

In [3]:
class DatasetVectorizer():
    
    """ a class to vectorize a pandas dataset using one of sklearn vectorizers """
    def __init__(self, vec):
        self.vectorizer = vec
        self.data = pd.DataFrame()
        self.train = pd.DataFrame()
        self.test = pd.DataFrame()
        self.X = 0
        self.X_test = 0
        self.y = 0
        self.y_test = 0
        
    def load_data(self, path):
        """ opens a .csv file using pandas dataframe """
        self.data = pd.read_csv(path)
        
    def train_test_split(self):
        """ splits data using sklearn train_test_split() function """
        self.train, self.test = train_test_split(self.data, test_size=0.1, shuffle=True)
        self.train.reset_index(inplace=True)
        self.test.reset_index(inplace=True)
        
    def vectorizeXy(self, feature_col, value_col):
        """ transforms a dataset into X, X_test, Y and Y_test """
        self.X = self.vectorizer.fit_transform(self.train[feature_col])
        self.X_test = self.vectorizer.transform(self.test[feature_col])
        self.y = self.train[value_col].values
        self.y_test = self.test[value_col].values

In [4]:
dv = DatasetVectorizer(TfidfVectorizer(min_df=3, max_df=0.5))
dv.load_data('data/labeled.csv')
dv.data.head()

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол, вон, а у хохлов еще хуже. Если бы хохлов не было, кисель их бы придумал.\n",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорбление, а доказанный факт - не-дебил про себя во множественном числе писать не будет. Или мы в тебя верим - это ты и твои воображаемые друзья?\n",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скрипалей отравила Россия? Анализировать и думать пытаешься? Ватник что ли?)\n",1.0


# Задание 2. 
Векторизовать можно любым способом, но постарайтесь, чтобы 1 и 0 не получались в близостях.
Результат запишите в тетрадке.


### а) Посчитайте близость между 3 и 12666 текстами в датасете (labeled.csv из семинара) 

In [5]:
# vectorize whole dataset
X = dv.vectorizer.fit_transform(dv.data.comment)
dv.data.loc[[3, 12666]]

,comment,toxic
3,"Страницу обнови, дебил. Это тоже не оскорбление, а доказанный факт - не-дебил про себя во множественном числе писать не будет. Или мы в тебя верим - это ты и твои воображаемые друзья?\n",1.0
12666,"Это не параноик, это дебил\n",1.0


In [6]:
print(1 - cosine_distances(X[3], X[12666]))
print(cosine_similarity(X[3], X[12666]))

[[0.5290017]]
[[0.5290017]]


### б) найдите 3 самых близких текста к тексту номер 43; выведите сами тексты и значения близостей, а не только индексы этих текстов. 

In [7]:
closest = cosine_distances(X[43], X)
print(closest.argsort()[0,:4])

[   43  1986 14325  5564]


In [8]:
# how close
print(1-closest[0,[43, 1986, 14325,  5564]])

[1.         0.20875876 0.20196174 0.1772219 ]


In [9]:
dv.data.loc[[43, 1986, 14325,  5564]]

,comment,toxic
43,"Люди зажрались и охуели если по мнению этих игроков андромеда лучше Антема. Хотя че там с багаутом76 сравнивают вон... Вот оно че оказывается, игроки просто охуели, ну ок. Вот долбанные пидерасы, не хотят покупать такую отличную игру, а еще смеют ругать такой божественный геймплей. Баги, хуевый геймплей поправят, а контент, ну его запилят, через год другой, причем бесплатно! Ага. Стоит лишь потерпеть!\n",1.0
1986,НУ И КАКАЯ МРАЗЬ КИДАЕТ ССЫЛКИ? ОХУЕЛИ ТАМ В КРАЙ УЖЕ?\n,1.0
14325,"Ну оль, ну вот че тебе надо? Иди в по сри, а этот достопочтенный тред оставь в покое\n",1.0
5564,"МАААМ, НУ ЧЕ ОНИ ЗА МНОЙ ХОДЯТ?\n",1.0


# Задание 3

Обучите 2 любых разных классификатора из семинара.

Предскажите токсичность для текстов из 2ch_corpus.txt (второй семинар) и найдите 10 самых токсичных для каждого из классификаторов. Сравните получаемые тексты - какие тексты совпадают, какие отличаются, правда ли тексты токсичные? *Можете предсказывать не все данные, но не менее 2500.

Требования к классификаторам:
а) один должен использовать CountVectorizer, другой TfidfVectorizer
б) у векторазера должны быть вручную заданы как минимум 5 параметров
в) у классификатора должно быть задано вручную как минимум 2 параметра
г)  f1 мера каждого из классификаторов должна быть минимум 0.75

In [10]:
stops = stopwords.words('russian')
dvach = open('data/2ch_corpus.txt', encoding="utf-8").readlines()[0:2500]

In [11]:
dv1 = DatasetVectorizer(TfidfVectorizer(min_df=3,
                                        max_df=0.4,
                                        max_features=5500,
                                        ngram_range=(1,1),
                                        stop_words=None
                                       ))
dv1.load_data('data/labeled.csv')
dv1.train_test_split()
dv1.vectorizeXy('comment', 'toxic')

### LogisticRegression

In [12]:

lr = LogisticRegression(C=1, class_weight='balanced')
lr.fit(dv1.X, dv1.y) # create a model
preds = lr.predict(dv1.X_test)
print(classification_report(dv1.y_test, preds))

              precision    recall  f1-score   support

         0.0       0.91      0.90      0.90       976
         1.0       0.79      0.82      0.80       466

    accuracy                           0.87      1442
   macro avg       0.85      0.86      0.85      1442
weighted avg       0.87      0.87      0.87      1442



In [13]:
probas = lr.predict_proba(dv1.X_test)
probas

array([[0.54923394, 0.45076606],
       [0.08783946, 0.91216054],
       [0.86775878, 0.13224122],
       ...,
       [0.54673297, 0.45326703],
       [0.61451653, 0.38548347],
       [0.10297403, 0.89702597]])

In [14]:
# check if probability of class 2 is higher than a given threshold
print(classification_report(dv1.y_test, (probas[:,1]>0.550).astype(int)))

              precision    recall  f1-score   support

         0.0       0.89      0.93      0.91       976
         1.0       0.83      0.77      0.80       466

    accuracy                           0.88      1442
   macro avg       0.86      0.85      0.85      1442
weighted avg       0.87      0.88      0.87      1442



In [15]:
dvach_vec = dv1.vectorizer.transform(dvach)
dvach_preds_lr = lr.predict(dvach_vec)
dvach_probas_lr = lr.predict_proba(dvach_vec)
print(dvach_preds_lr.shape, dvach_probas_lr.shape)

(2500,) (2500, 2)


In [16]:
dvach_lr_labeled = pd.DataFrame({"comment": dvach, 'toxic': dvach_preds_lr, 'prob_tox' : dvach_probas_lr[:, 1]})

### MultinomialNB

In [17]:
dv2 = DatasetVectorizer(CountVectorizer(min_df=3,
                                        max_df=0.4,
                                        max_features=5000,
                                        ngram_range=(1,1),
                                        stop_words=None))
dv2.load_data('data/labeled.csv')
dv2.train_test_split()
dv2.vectorizeXy('comment', 'toxic')

nb = MultinomialNB(alpha=2., fit_prior=False)
nb.fit(dv2.X, dv2.y)
preds = nb.predict(dv2.X_test)

print(classification_report(dv2.y_test, preds))

              precision    recall  f1-score   support

         0.0       0.90      0.90      0.90       972
         1.0       0.79      0.80      0.79       470

    accuracy                           0.86      1442
   macro avg       0.84      0.85      0.85      1442
weighted avg       0.86      0.86      0.86      1442



In [18]:
dvach_vec = dv2.vectorizer.transform(dvach)
dvach_preds_nb = nb.predict(dvach_vec)
dvach_probas_nb = nb.predict_proba(dvach_vec)
print(dvach_preds_nb.shape, dvach_probas_nb.shape)

(2500,) (2500, 2)


In [19]:
dvach_nb_labeled = pd.DataFrame({"comment": dvach, 'toxic': dvach_preds_nb, 'prob_tox' : dvach_probas_nb[:, 1]})

### Сравнение результатов

In [20]:
dvach_lr_labeled.sort_values('prob_tox',ascending=False).head(10)

,comment,toxic,prob_tox
2463,Крысу ты разделовал?\n,1.0,0.998815
87,"Ты превращаешься в Суисейсеки, десу.\n",1.0,0.998815
271,>ElixirНу ты и говноед.\n,1.0,0.998815
895,"Лол. Как школьник: его послали, а он: ""нет ты нахуй!""\n",1.0,0.997900
2077,">на окраине поселка Ты, блять, колхозник.\n",1.0,0.997357
2255,А ты и не станешь. Лол.\n,1.0,0.996438
1584,Ты и собаке отсосал бы\n,1.0,0.995976
118,оуоуоуо уча уча уча аиавивива ну ты ну ты ну ты ВИЧУУУУУУУУУУУУУУУУУХА\n,1.0,0.993823
938,Ебать ты юморист донный.\n,1.0,0.993739
1775,"Кто подданный? Ты подданый, ёпт. Здравствуй.\n",1.0,0.993461


In [21]:
dvach_nb_labeled.sort_values('prob_tox',ascending=False).head(10)

,comment,toxic,prob_tox
645,"Ёбаные советские названия блять. КРУЖКИ блять. Всегда ненавидел. Блять. Почему бы не сказать КЛУБ ПО ИНТЕРЕСАМ, как у япошек, нет блять, будем гуманитарную хуйню, сравнения блять, типа В КРУГ СОБРАЛИСЬ, ахахахаха кружок ахахахахаха))))) пиздец блять. Может при Сталине это звучало, но уже при Горбачёве это просто бесило, а сейчас это выглядит вообще как атавизм. Или ещё ебанутое слово СЛЁТ блять. Сука, СЛЁТ, почему слёт, а не съезд? Типа съезд - это для КПСС, не доросли ещё? Или типа в Совке...",1.0,1.0
1674,"Как олдфаг 29лвл, заставший конец двача, прошедший весь педальчан, и вкусивший сполна абучана, могу заверить: раньше аудиория была другой. Особенно круто было на дваче и раннем педальчане, ибо преобладала студентота преимущественно из технарей, гики, задроты, анимешники, поехавшие, и прочие интересные личности. Конечно, было дохера тредов про ""двач я посрал"", или тупые перефорсы с упячки, форчана, да даже с удаффа залетали и срали. Но это было иначе, чувствовалось, что говно, которое ты куш...",1.0,1.0
256,">Scala, Clojure, Erlang, Elixir, Common Lisp, Haskell, Ocaml, F#, Elm, Swift, Go, Rust, D, Nim, Scheme, SmalltalkМолодец, перечислил сложные для старта языки + на которых вакансий хуй хуй да нихуя. Сам-то с какого языка начинал? И наверное в гарварде и mit дауны сидят и учат людей сям и питону, сука на дваче, что не доска, так куча нитаких, как все даунов на хайпе, которые толкают свое говно. Нахуй иди.\n",1.0,1.0
2467,Иди нахуй дурачек ебаный пидорас ну чтоьты за скотина ебаная оп выше написал что нашел ее а теперь иди нахкй\n,1.0,1.0
2260,"Не ты ли в мотаче про трек-сессии рассказывал?Ну и потом, как бы то ни было, нахуй тебе всралась жена, которой на тебя похуй. Ей на тебя похуй, понимаешь?\n",1.0,1.0
456,"мамка твоя мне заработала своей пиздой вонючей тварь вонючая, мразь поганая, гандон, говно, пидрила, ану иди сюда\n",1.0,1.0
2395,"Сука, чуть не плюнул винищем в монитор. Сука, не траль, не траль, сука. Таких тупых как ты не бывает, ебучая моль.\n",1.0,1.0
492,"А знаете, я расскажу одну историю. У меня, пару лет назад, когда я был школьником, был сосед. Владиком звать. История типичная: омега, травля, всратый и молчаливый. Я с ним пытался сдружиться, но он был пассивен к этому. Его мать удивлялась, мол ПОЧЕМУ МОЮ КОРЗИНКУ НИКТО НЕ ЛЮБИТ, НИКТО НЕ ДРУЖИТ С НИМ. Но с ним никакого смысла дружить нет! Он не может поддержать разговор, шутит тупые шутки и страшный на вид из-за усов. Это всё я веду к тому, что тебе, ОП не стоит удивляться отсутствию круг...",1.0,1.0
704,"Набежали хохлы дауны которы не смогли решить задачу, обиделись, и во всем виноват совок) хохлы - такие хохлы\n",1.0,1.0
752,"ммм, из разряда "" у тебя так бомбит потому что ты не хохол"" где связь между мясными хохлами которых пиздят и евровидением? Я эту хуйню никогда не смотрел и вообще похуй на нее\n",1.0,1.0


Пересечений в топ-10 не надблюдается. Оба классификатора выдают, за некоторым исключением, подходящие примеры.